In [14]:
from astroquery.vizier import Vizier
from astropy.table import Table
from astropy import units as u
import pandas as pd
import numpy as np

In [10]:
single_star = 'proxima centauri'

column_mass = Vizier(columns=['Mass'])
tables_mass = column_mass.query_object(single_star, radius= 2 *u.arcsec)

print(tables_mass)

TableList with 27 tables:
	'0:IV/38/tic' with 1 column(s) and 3 row(s) 
	'1:IV/39/tic82' with 1 column(s) and 3 row(s) 
	'2:J/ApJ/834/17/table1' with 1 column(s) and 1 row(s) 
	'3:J/ApJ/834/85/table1' with 1 column(s) and 1 row(s) 
	'4:J/ApJ/890/23/table1' with 1 column(s) and 1 row(s) 
	'5:J/ApJ/891/58/table6' with 1 column(s) and 1 row(s) 
	'6:J/ApJ/902/115/table1' with 1 column(s) and 1 row(s) 
	'7:J/ApJS/247/11/table5' with 1 column(s) and 1 row(s) 
	'8:J/ApJS/266/6/table3' with 1 column(s) and 1 row(s) 
	'9:J/A+A/505/859/targets' with 1 column(s) and 1 row(s) 
	'10:J/A+A/551/A36/table2' with 1 column(s) and 1 row(s) 
	'11:J/A+A/600/A13/tablea3' with 1 column(s) and 1 row(s) 
	'12:J/A+A/624/A94/tablea1' with 1 column(s) and 1 row(s) 
	'13:J/A+A/638/A20/tableb1' with 1 column(s) and 1 row(s) 
	'14:J/A+A/640/A83/table' with 1 column(s) and 5 row(s) 
	'15:J/A+A/644/A68/tablea1' with 1 column(s) and 1 row(s) 
	'16:J/A+A/661/A23/tableb1' with 1 column(s) and 1 row(s) 
	'17:J/A+A/675/A16

In [15]:
# Read the Excel file
excel_file = 'Mass_stars_sample.xlsx'
sheet_name = 'sample'  
column_name = 'target'  

# Load the data from the Excel file
data = pd.read_excel(excel_file, sheet_name=sheet_name)

# Extract stars from the specified column
stars = data[column_name].tolist()

#Check if you list of stars is correct
print(stars)

['proxima centauri', 'sirius']


In [22]:
combined_table = Table(names=['Target', 'Mass','s_Mass', 'Mass unit','Description', 'id table', 'id catalog'],
                      dtype =('U100', 'f', 'f', 'U20', 'U100', 'U100', 'U100'))

for star in stars:
    star_mass = Vizier(columns=['Mass', 's_Mass'])
    tables = star_mass.query_object(star, radius=2*u.arcsec)
    
    if not tables:
        combined_table.add_row([str(star), np.nan, np.nan, str(np.nan), str(np.nan), str(np.nan), str(np.nan)])
        
    else:
        description_id = str(tables)
        table_names = [line.split()[0][1:-1] for line in description_id.split("\n")[1:]]
        id_table = [name.split(':')[1].strip() for name in table_names]
        id_catalog = ["/".join(item.split("/")[:-1]) for item in id_table]
        
        for i, table in enumerate(tables):
            
            for row in table:
                mass_value = row['Mass']
                uncertainty_value = row['s_Mass'] if 's_Mass' in row.colnames else np.nan
                
                if mass_value and mass_value != '--':
                    mass = float(mass_value)
                else: 
                    mass = np.nan
                    
                if uncertainty_value and uncertainty_value != '--':
                    uncertainty_mass = float(uncertainty_value)
                else:
                    uncertainty_mass = np.nan
                
                each_table = tables[i]
                unit_mass = str(each_table['Mass'].unit)
                description = str(each_table['Mass'].description)
                
                combined_table.add_row([str(star), mass, uncertainty_mass, unit_mass, 
                                        description, id_table[i], id_catalog[i]])
                
print(combined_table)

     Target        Mass  s_Mass ...         id table            id catalog   
---------------- ------- ------ ... ------------------------ ----------------
proxima centauri     nan    nan ...                IV/38/tic            IV/38
proxima centauri     nan    nan ...                IV/38/tic            IV/38
proxima centauri   0.126   0.02 ...                IV/38/tic            IV/38
proxima centauri     nan    nan ...              IV/39/tic82            IV/39
proxima centauri     nan    nan ...              IV/39/tic82            IV/39
proxima centauri   0.126   0.02 ...              IV/39/tic82            IV/39
proxima centauri   0.118    nan ...      J/ApJ/834/17/table1     J/ApJ/834/17
proxima centauri   0.119    nan ...      J/ApJ/834/85/table1     J/ApJ/834/85
proxima centauri    0.12    nan ...      J/ApJ/890/23/table1     J/ApJ/890/23
proxima centauri 0.12563    nan ...      J/ApJ/891/58/table6     J/ApJ/891/58
             ...     ...    ... ...                      ...    

In [23]:
combined_table

Target,Mass,s_Mass,Mass unit,Description,id table,id catalog
str100,float32,float32,str20,str100,str100,str100
proxima centauri,nan,nan,solMass,[0.08/3.3]? Mass (20),IV/38/tic,IV/38
proxima centauri,nan,nan,solMass,[0.08/3.3]? Mass (20),IV/38/tic,IV/38
proxima centauri,0.126,0.02,solMass,[0.08/3.3]? Mass (20),IV/38/tic,IV/38
proxima centauri,nan,nan,solMass,[0.08/3.3]? Mass (17),IV/39/tic82,IV/39
proxima centauri,nan,nan,solMass,[0.08/3.3]? Mass (17),IV/39/tic82,IV/39
proxima centauri,0.126,0.02,solMass,[0.08/3.3]? Mass (17),IV/39/tic82,IV/39
proxima centauri,0.118,nan,None,[0.0003/318] Mass,J/ApJ/834/17/table1,J/ApJ/834/17
proxima centauri,0.119,nan,solMass,[0.05/0.9] Stellar mass,J/ApJ/834/85/table1,J/ApJ/834/85
proxima centauri,0.12,nan,solMass,[0.01/23.6]? Stellar mass,J/ApJ/890/23/table1,J/ApJ/890/23


In [20]:
# Convert the Astropy table to a Pandas DataFrame
combined_df = combined_table.to_pandas()

# Specify the file path where you want to save the Excel file
excel_file_path = 'Mass_stars.xlsx'

# Save the DataFrame to an Excel file
combined_df.to_excel(excel_file_path, sheet_name='test', index=False)

print(f"Excel file '{excel_file_path}' has been saved.")

Excel file 'Mass_stars_sample.xlsx' has been saved.
